### Imports

In [34]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
pd.options.display.max_rows = 1000  # Adjust as needed


### Dataset

In [45]:

# Data
plants = ['Chicago', 'Princeton', 'Atlanta', 'LosAngeles']
regions = ['Northwest', 'Southwest', 'UpperMidwest', 'LowerMidwest', 'Northeast', 'Southeast']
products = ['Wipes', 'Ointments']


# Fixed costs for plants and lines
fixed_costs = {
    'Chicago': {'Wipes': 4500000, 'Ointments': 2000000},
    'Princeton': {'Wipes': 2200000, 'Ointments': 1700000},
    'Atlanta': {'Wipes': 3200000, 'Ointments': 1500000},
    'LosAngeles': {'Wipes': 2500000, 'Ointments': 1500000}
}

# Variable production costs
variable_costs = {'Wipes': 10, 'Ointments': 20}

# Transportation costs
transport_costs = {
    'Chicago': [6.32, 6.32, 3.68, 4.04, 5.76, 5.96],
    'Princeton': [6.6, 6.6, 5.76, 5.92, 3.68, 4.08],
    'Atlanta': [6.72, 6.48, 5.92, 4.08, 4.04, 3.64],
    'LosAngeles': [4.36, 3.68, 6.32, 6.32, 6.72, 6.6]
}

# Demand in units
demand = {
        'Wipes': [500000, 670000, 900000, 800000, 1000000, 600000],
        'Ointments': [50000, 110000, 120000, 70000, 120000, 70000]
}

# Capacity in units
capacity = {
    'Wipes': {
        'Chicago': 5000000, 
        'Princeton': 2000000, 
        'Atlanta': 2000000, 
        'LosAngeles': 2000000
    },
    'Ointments': {
        'Chicago': 1000000, 
        'Princeton': 1000000, 
        'Atlanta': 1000000, 
        'LosAngeles': 1000000
    }
}

# Modify transportation costs for each scenario
transport_costs_half = {p: [cost / 2 for cost in costs] for p, costs in transport_costs.items()}
transport_costs_double = {p: [cost * 2 for cost in costs] for p, costs in transport_costs.items()}

# Minimum production requirements for Chicago
min_production = {'Wipes': 1, 'Ointments': 1}  # Replace '1' with appropriate non-zero minimum values

# Adjusted demand with 35% hike
demand_hiked = {
    'Wipes': [int(d * 1.35) for d in demand['Wipes']],
    'Ointments': [int(d * 1.35) for d in demand['Ointments']]
}

### Question 1

Calculate the Annual Cost of Serving the Nation from Chicago

Total Cost=Fixed Costs+Variable Costs+Transportation Costs

In [46]:
# Data for calculation
fixed_cost_chicago = fixed_costs['Chicago']['Wipes'] + fixed_costs['Chicago']['Ointments']

# Variable Costs
variable_cost_chicago = (
    sum(demand['Wipes']) * variable_costs['Wipes'] +
    sum(demand['Ointments']) * variable_costs['Ointments']
)

# Transportation Costs
# Calculating transportation costs region-wise for both products
transport_cost_chicago = sum(
    demand['Wipes'][i] * transport_costs['Chicago'][i] + 
    demand['Ointments'][i] * transport_costs['Chicago'][i]
    for i in range(len(regions))
)

# Total cost from Chicago
total_cost_chicago = fixed_cost_chicago + variable_cost_chicago + transport_cost_chicago

print("Fixed Costs for Chicago:", fixed_cost_chicago)
print("Variable Costs for Chicago:", variable_cost_chicago)
print("Transport Costs for Chicago:", transport_cost_chicago)
print("Total Annual Cost of Serving from Chicago:", total_cost_chicago)


Fixed Costs for Chicago: 6500000
Variable Costs for Chicago: 55500000
Transport Costs for Chicago: 26118400.0
Total Annual Cost of Serving from Chicago: 88118400.0


### Define Function

In [63]:
def solve_coolwipes_optimization(fixed_costs, variable_costs, transport_costs, demand, capacity, min_production, plants, regions, products, include_chicago_constraints=True):
    # Create the optimization model
    model = Model("CoolWipes_Optimization")
    # Suppress detailed output
    model.setParam("OutputFlag", 0)  # No output from the solver
    # Decision Variables
    X = model.addVars(
        [(p, r, k) for p in plants for r in regions for k in products],
        name="Production",
        vtype=GRB.CONTINUOUS,
        lb=0
    )
    Y = model.addVars(
        [(p, k) for p in plants for k in products],
        name="PlantActive",
        vtype=GRB.BINARY
    )

    # Objective Function: Minimize total cost
    model.setObjective(
        quicksum(fixed_costs[p][k] * Y[p, k] for p in plants for k in products) +
        quicksum(variable_costs[k] * X[p, r, k] for p in plants for r in regions for k in products) +
        quicksum(transport_costs[p][i] * X[p, regions[i], k]
                 for p in plants for i in range(len(regions)) for k in products),
        GRB.MINIMIZE
    )

    # Constraints: Demand satisfaction
    for r, region in enumerate(regions):
        for k in products:
            model.addConstr(
                quicksum(X[p, region, k] for p in plants) >= demand[k][r],
                name=f"Demand_{region}_{k}"
            )

    # Constraints: Capacity limits
    for p in plants:
        for k in products:
            model.addConstr(
                quicksum(X[p, r, k] for r in regions) <= capacity[k][p] * Y[p, k],
                name=f"Capacity_{p}_{k}"
            )

    if include_chicago_constraints:
        # Ensure Chicago is always active
        for k in products:
            model.addConstr(Y['Chicago', k] == 1, name=f"Chicago_Always_Active_{k}")

        # Ensure Chicago produces at least a minimum level
        for k in products:
            model.addConstr(
                quicksum(X['Chicago', r, k] for r in regions) >= min_production[k],
                name=f"Chicago_Min_Production_{k}"
            )

    # Solve the model
    model.optimize()

    # Results
    if model.status == GRB.OPTIMAL:
        print("\nOptimal Plant Configuration and Costs:")
        for p in plants:
            for k in products:
                if Y[p, k].x > 0.5:
                    print(f"Plant {p} produces {k}: Activated")
        print("\nProduction Plan:")
        for p in plants:
            for r in regions:
                for k in products:
                    if X[p, r, k].x > 0:
                        print(f"{p} supplies {X[p, r, k].x:.2f} units of {k} to {r}")
        print("\nTotal Cost:", round(model.objVal, 2))

        # Capacity Utilization Report
        print("\nCapacity Utilization Report:")
        for p in plants:
            for k in products:
                total_production = sum(X[p, r, k].x for r in regions if X[p, r, k].x > 0)
                if total_production > 0:
                    utilization = (total_production / capacity[k][p]) * 100 if capacity[k][p] > 0 else 0
                    print(f"Plant {p}, Product {k}: Utilization = {round(utilization, 2)}%")
                    if utilization > 100:
                        print(f"Warning: Plant {p} exceeds capacity for {k}!")
    else:
        print("No optimal solution found.")


### Question 2A

**Minimum Production Requirements**
A minimum production requirement is set for the Chicago plant to ensure it operates at a non-zero utilization level, even if operating below full capacity:
\[
\sum_r X[\text{Chicago}, r, k] \geq \text{min\_production}[k], \; \forall k
\]

This ensures Chicago produces at least a small amount of each product while still allowing flexibility for cost minimization.

---

**Objective Function**
The primary goal of the optimization is to **minimize the total cost**, which includes:

1. **Fixed Costs**: The cost of activating production lines at plants.
   - \( \text{FixedCost}_{p,k} \cdot Y[p,k] \): Adds fixed cost if a plant \( p \) is active for product \( k \).
2. **Variable Costs**: The cost of producing each unit of wipes and ointments.
   - \( \text{VariableCost}_k \cdot X[p,r,k] \): Adds production costs proportional to the number of units produced.
3. **Transportation Costs**: The cost of shipping products from plants to regions.
   - \( \text{TransportCost}_{p,r,k} \cdot X[p,r,k] \): Adds costs for transporting products from plant \( p \) to region \( r \).

---

**Decision Variables**
1. \( X[p, r, k] \): Continuous variable representing the number of units of product \( k \) (wipes or ointments) produced at plant \( p \) and sent to region \( r \).
   - Range: \( 0 \leq X[p, r, k] \), meaning production and shipment quantities cannot be negative.
2. \( Y[p, k] \): Binary variable indicating whether plant \( p \) is active for product \( k \).
   - \( Y[p, k] \in \{0, 1\} \), meaning a plant is either active (\( Y[p, k] = 1 \)) or inactive (\( Y[p, k] = 0 \)).

---

**Constraints**

**Demand Satisfaction**
Ensures that the total supply from all plants meets or exceeds the demand for each product in each region:
\[
\sum_p X[p, r, k] \geq \text{Demand}[r, k], \; \forall r, k
\]

**Purpose**:
- Ensures that regional demands for wipes and ointments are fully satisfied.

---

**Capacity Limits**
Ensures that the total production at a plant for a product does not exceed its capacity if the plant is active:
\[
\sum_r X[p, r, k] \leq \text{Capacity}[p, k] \cdot Y[p, k], \; \forall p, k
\]

**Purpose**:
- Limits production at each plant based on its maximum capacity.
- Ties production to the activation status of the plant:
  - If \( Y[p, k] = 1 \): The plant can produce up to its capacity.
  - If \( Y[p, k] = 0 \): The plant cannot produce at all (\( X[p, r, k] = 0 \)).

---

**Ensure Chicago is Always Active**
Guarantees that Chicago's production lines for wipes and ointments are always active:
\[
Y[\text{Chicago}, k] = 1, \; \forall k
\]

**Purpose**:
- Ensures that Chicago's production lines remain operational, regardless of the cost-efficiency of other plants.

---

**Minimum Production for Chicago**
Ensures that Chicago produces at least a minimum non-zero amount for both wipes and ointments:
\[
\sum_r X[\text{Chicago}, r, k] \geq \text{min\_production}[k], \; \forall k
\]

**Purpose**:
- Enforces non-zero utilization for Chicago's production lines while still allowing flexibility for cost minimization.

---

**Summary of Problem Steps Addressed**
| **Step**                         | **Description**                                                                                  | **Code Reference**                                                                 |
|-----------------------------------|--------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------|
| **Objective Function**            | Minimize fixed costs, variable costs, and transportation costs.                                 | `model.setObjective(...)`                                                         |
| **Decision Variables**            | \( X[p, r, k] \) (production and shipping) and \( Y[p, k] \) (plant activation).                | `model.addVars(...)`                                                              |
| **Demand Satisfaction**           | Total supply to each region meets demand for wipes and ointments.                               | `model.addConstr(...)` (Demand constraints)                                       |
| **Capacity Limits**               | Production at each plant does not exceed its capacity if the plant is active.                   | `model.addConstr(...)` (Capacity constraints)                                     |
| **Chicago Always Active**         | Chicago's production lines are always active, ensuring the plant is utilized.                   | `model.addConstr(Y['Chicago', k] == 1)`                                           |
| **Minimum Production for Chicago**| Enforces non-zero production levels for Chicago’s wipes and ointment lines.                    | `model.addConstr(...)` (Minimum production constraints)                           |


In [64]:
# Call with Chicago constraints
solve_coolwipes_optimization(
    fixed_costs=fixed_costs,
    variable_costs=variable_costs,
    transport_costs=transport_costs,
    demand=demand,
    capacity=capacity,
    min_production=min_production,
    plants=plants,
    regions=regions,
    products=products,
    include_chicago_constraints=True  # Set to False to exclude Chicago constraints
)



Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Chicago produces Ointments: Activated
Plant Princeton produces Wipes: Activated
Plant LosAngeles produces Wipes: Activated

Production Plan:
Chicago supplies 50000.00 units of Ointments to Northwest
Chicago supplies 110000.00 units of Ointments to Southwest
Chicago supplies 900000.00 units of Wipes to UpperMidwest
Chicago supplies 120000.00 units of Ointments to UpperMidwest
Chicago supplies 800000.00 units of Wipes to LowerMidwest
Chicago supplies 70000.00 units of Ointments to LowerMidwest
Chicago supplies 120000.00 units of Ointments to Northeast
Chicago supplies 70000.00 units of Ointments to Southeast
Princeton supplies 1000000.00 units of Wipes to Northeast
Princeton supplies 600000.00 units of Wipes to Southeast
LosAngeles supplies 500000.00 units of Wipes to Northwest
LosAngeles supplies 670000.00 units of Wipes to Southwest

Total Cost: 86861600.0

Capacity Utilization Report:
Plant Chicago, 

### Question 2B

In [65]:
# Example call to solve_coolwipes_optimization with halved transportation costs
solve_coolwipes_optimization(
    fixed_costs=fixed_costs,
    variable_costs=variable_costs,
    transport_costs=transport_costs_half,  # Halved transportation costs
    demand=demand,
    capacity=capacity,
    min_production=min_production,  # Minimum production for Chicago
    plants=plants,
    regions=regions,
    products=products,
    include_chicago_constraints=True  # Ensure Chicago constraints are included
)



Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Chicago produces Ointments: Activated

Production Plan:
Chicago supplies 500000.00 units of Wipes to Northwest
Chicago supplies 50000.00 units of Ointments to Northwest
Chicago supplies 670000.00 units of Wipes to Southwest
Chicago supplies 110000.00 units of Ointments to Southwest
Chicago supplies 900000.00 units of Wipes to UpperMidwest
Chicago supplies 120000.00 units of Ointments to UpperMidwest
Chicago supplies 800000.00 units of Wipes to LowerMidwest
Chicago supplies 70000.00 units of Ointments to LowerMidwest
Chicago supplies 1000000.00 units of Wipes to Northeast
Chicago supplies 120000.00 units of Ointments to Northeast
Chicago supplies 600000.00 units of Wipes to Southeast
Chicago supplies 70000.00 units of Ointments to Southeast

Total Cost: 75059200.0

Capacity Utilization Report:
Plant Chicago, Product Wipes: Utilization = 89.4%
Plant Chicago, Product Ointments: Utilization = 54.0%


### Question 2C

In [66]:
# Example call to solve_coolwipes_optimization with halved transportation costs
solve_coolwipes_optimization(
    fixed_costs=fixed_costs,
    variable_costs=variable_costs,
    transport_costs=transport_costs_double,  # Double transportation costs
    demand=demand,
    capacity=capacity,
    min_production=min_production,  # Minimum production for Chicago
    plants=plants,
    regions=regions,
    products=products,
    include_chicago_constraints=True  # Ensure Chicago constraints are included
)



Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Chicago produces Ointments: Activated
Plant Princeton produces Wipes: Activated
Plant LosAngeles produces Wipes: Activated

Production Plan:
Chicago supplies 50000.00 units of Ointments to Northwest
Chicago supplies 110000.00 units of Ointments to Southwest
Chicago supplies 900000.00 units of Wipes to UpperMidwest
Chicago supplies 120000.00 units of Ointments to UpperMidwest
Chicago supplies 800000.00 units of Wipes to LowerMidwest
Chicago supplies 70000.00 units of Ointments to LowerMidwest
Chicago supplies 120000.00 units of Ointments to Northeast
Chicago supplies 70000.00 units of Ointments to Southeast
Princeton supplies 1000000.00 units of Wipes to Northeast
Princeton supplies 600000.00 units of Wipes to Southeast
LosAngeles supplies 500000.00 units of Wipes to Northwest
LosAngeles supplies 670000.00 units of Wipes to Southwest

Total Cost: 107023200.0

Capacity Utilization Report:
Plant Chicago,

### Question 3A

**Problem Explanation**

**A. Designing a New Network from Scratch**
If Matt could design a new production network from scratch, including the Chicago plant as a new option, the goal would be to identify the most cost-effective network. This involves:
1. Treating Chicago as a new optional plant (not mandatory).
2. Allowing Princeton, Atlanta, Los Angeles, and Chicago to activate production lines if cost-effective.
3. Solving for the production network that minimizes total costs.

**B. Impact of Halved Transportation Costs**
1. If transportation costs are halved, shipping products from centralized plants (like Chicago) to distant regions becomes cheaper.
2. The optimization may prioritize centralized production by activating fewer plants and utilizing Chicago or other large-capacity plants.

**C. Impact of Doubled Transportation Costs**
1. If transportation costs are doubled, shipping becomes expensive, especially for distant regions.
2. The optimization will favor decentralized production by activating plants closer to demand centers, reducing transportation costs at the expense of higher fixed costs for additional plants.

---

**Steps to Solve**

**1. Objective Function**
Minimize the total cost, which includes:
1. **Fixed Costs**:
   - The cost of activating production lines at plants.
   \[
   \text{FixedCost}_{p,k} \cdot Y[p,k]
   \]
2. **Variable Costs**:
   - The cost of producing each unit of wipes or ointments.
   \[
   \text{VariableCost}_k \cdot X[p,r,k]
   \]
3. **Transportation Costs**:
   - The cost of shipping products from plants to regions.
   \[
   \text{TransportCost}_{p,r,k} \cdot X[p,r,k]
   \]

---

**2. Decision Variables**
1. \( X[p, r, k] \): Continuous variable representing the amount of product \( k \) produced at plant \( p \) and shipped to region \( r \).
   - Range: \( 0 \leq X[p, r, k] \).
2. \( Y[p, k] \): Binary variable indicating whether plant \( p \) is active for product \( k \).
   - \( Y[p, k] \in \{0, 1\} \).

---

**3. Constraints**
1. **Demand Satisfaction**:
   Ensure total supply to each region meets or exceeds the demand:
   \[
   \sum_p X[p, r, k] \geq \text{Demand}[r, k], \; \forall r, k
   \]

2. **Capacity Limits**:
   Ensure production at each plant does not exceed its capacity:
   \[
   \sum_r X[p, r, k] \leq \text{Capacity}[p, k] \cdot Y[p, k], \; \forall p, k
   \]

3. **Binary Plant Activation**:
   Enforce binary constraints on plant activation:
   \[
   Y[p, k] \in \{0, 1\}, \; \forall p, k
   \]

---

**Scenarios to Solve**

**Scenario A: Default Transportation Costs**
- Solve with the transportation costs as provided in the dataset.
- The solver will decide which plants to activate and how to distribute production to minimize costs.

**Scenario B: Halved Transportation Costs**
- Reduce transportation costs by 50%.
- Expect centralized production with fewer plants activated, especially plants with higher capacities like Chicago.

**Scenario C: Doubled Transportation Costs**
- Double the transportation costs.
- Expect decentralized production with more regional plants activated to minimize shipping distances and transportation costs.

---

**Summary of Problem Steps Addressed**
| **Step**                         | **Description**                                                                                  | **Code Reference**                                                                 |
|-----------------------------------|--------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------|
| **Objective Function**            | Minimize fixed costs, variable costs, and transportation costs.                                 | `model.setObjective(...)`                                                         |
| **Decision Variables**            | \( X[p, r, k] \) (production and shipping) and \( Y[p, k] \) (plant activation).                | `model.addVars(...)`                                                              |
| **Demand Satisfaction**           | Total supply to each region meets demand for wipes and ointments.                               | `model.addConstr(...)` (Demand constraints)                                       |
| **Capacity Limits**               | Production at each plant does not exceed its capacity if the plant is active.                   | `model.addConstr(...)` (Capacity constraints)                                     |
| **Plant Activation**              | Plants are activated only if cost-effective.                                                    | `model.addConstr(...)` (Binary constraints)                                       |
| **Scenario Analysis**             | Solve for default, halved, and doubled transportation costs to evaluate the network's flexibility. | Code structure for each transportation cost scenario.                             |


In [67]:
# Solve for default transportation costs
print("\nScenario A: Default Transportation Costs")
solve_coolwipes_optimization(
    fixed_costs=fixed_costs,
    variable_costs=variable_costs,
    transport_costs=transport_costs,
    demand=demand,
    capacity=capacity,
    min_production=min_production,
    plants=plants,
    regions=regions,
    products=products,
    include_chicago_constraints=False  # Chicago constraints disabled
)


Scenario A: Default Transportation Costs

Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Princeton produces Wipes: Activated
Plant Atlanta produces Ointments: Activated
Plant LosAngeles produces Wipes: Activated

Production Plan:
Chicago supplies 900000.00 units of Wipes to UpperMidwest
Chicago supplies 800000.00 units of Wipes to LowerMidwest
Princeton supplies 1000000.00 units of Wipes to Northeast
Princeton supplies 600000.00 units of Wipes to Southeast
Atlanta supplies 50000.00 units of Ointments to Northwest
Atlanta supplies 110000.00 units of Ointments to Southwest
Atlanta supplies 120000.00 units of Ointments to UpperMidwest
Atlanta supplies 70000.00 units of Ointments to LowerMidwest
Atlanta supplies 120000.00 units of Ointments to Northeast
Atlanta supplies 70000.00 units of Ointments to Southeast
LosAngeles supplies 500000.00 units of Wipes to Northwest
LosAngeles supplies 670000.00 units of Wipes to Southwest

Total Cost: 86302000.0

Ca

### Question 3B

In [68]:
# Solve for halved transportation costs
print("\nScenario B: Halved Transportation Costs")
solve_coolwipes_optimization(
    fixed_costs=fixed_costs,
    variable_costs=variable_costs,
    transport_costs=transport_costs_half,  # Halved transportation costs
    demand=demand,
    capacity=capacity,
    min_production=min_production,
    plants=plants,
    regions=regions,
    products=products,
    include_chicago_constraints=False  # Chicago constraints disabled
)



Scenario B: Halved Transportation Costs

Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Atlanta produces Ointments: Activated

Production Plan:
Chicago supplies 500000.00 units of Wipes to Northwest
Chicago supplies 670000.00 units of Wipes to Southwest
Chicago supplies 900000.00 units of Wipes to UpperMidwest
Chicago supplies 800000.00 units of Wipes to LowerMidwest
Chicago supplies 1000000.00 units of Wipes to Northeast
Chicago supplies 600000.00 units of Wipes to Southeast
Atlanta supplies 50000.00 units of Ointments to Northwest
Atlanta supplies 110000.00 units of Ointments to Southwest
Atlanta supplies 120000.00 units of Ointments to UpperMidwest
Atlanta supplies 70000.00 units of Ointments to LowerMidwest
Atlanta supplies 120000.00 units of Ointments to Northeast
Atlanta supplies 70000.00 units of Ointments to Southeast

Total Cost: 74529400.0

Capacity Utilization Report:
Plant Chicago, Product Wipes: Utilization = 89.4%
Plant Atlanta, Prod

### Question 3C

In [69]:
# Solve for doubled transportation costs 
print("\nScenario C: Transportation Costs Doubled")
solve_coolwipes_optimization(
    fixed_costs=fixed_costs,
    variable_costs=variable_costs,
    transport_costs=transport_costs_double,  # Doubled transportation costs
    demand=demand,
    capacity=capacity,
    min_production=min_production,
    plants=plants,
    regions=regions,
    products=products,
    include_chicago_constraints=False  # Chicago constraints disabled
)



Scenario C: Transportation Costs Doubled

Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Princeton produces Wipes: Activated
Plant Atlanta produces Ointments: Activated
Plant LosAngeles produces Wipes: Activated

Production Plan:
Chicago supplies 900000.00 units of Wipes to UpperMidwest
Chicago supplies 800000.00 units of Wipes to LowerMidwest
Princeton supplies 1000000.00 units of Wipes to Northeast
Princeton supplies 600000.00 units of Wipes to Southeast
Atlanta supplies 50000.00 units of Ointments to Northwest
Atlanta supplies 110000.00 units of Ointments to Southwest
Atlanta supplies 120000.00 units of Ointments to UpperMidwest
Atlanta supplies 70000.00 units of Ointments to LowerMidwest
Atlanta supplies 120000.00 units of Ointments to Northeast
Atlanta supplies 70000.00 units of Ointments to Southeast
LosAngeles supplies 500000.00 units of Wipes to Northwest
LosAngeles supplies 670000.00 units of Wipes to Southwest

Total Cost: 106404000.0

C

### Optimized for forecasted demand

In [71]:
 print(f"\nEvaluating Model with Expected Demand")
 solve_coolwipes_optimization(
        fixed_costs=fixed_costs,
        variable_costs=variable_costs,
        transport_costs=transport_costs,  # Use default transport costs for now
        demand=demand_hiked,
        capacity=capacity,
        min_production=min_production,
        plants=plants,
        regions=regions,
        products=products,
        include_chicago_constraints=True
    )


Evaluating Model with Expected Demand

Optimal Plant Configuration and Costs:
Plant Chicago produces Wipes: Activated
Plant Chicago produces Ointments: Activated
Plant Princeton produces Wipes: Activated
Plant LosAngeles produces Wipes: Activated

Production Plan:
Chicago supplies 67500.00 units of Ointments to Northwest
Chicago supplies 148500.00 units of Ointments to Southwest
Chicago supplies 1215000.00 units of Wipes to UpperMidwest
Chicago supplies 162000.00 units of Ointments to UpperMidwest
Chicago supplies 1080000.00 units of Wipes to LowerMidwest
Chicago supplies 94500.00 units of Ointments to LowerMidwest
Chicago supplies 162000.00 units of Ointments to Northeast
Chicago supplies 160000.00 units of Wipes to Southeast
Chicago supplies 94500.00 units of Ointments to Southeast
Princeton supplies 1350000.00 units of Wipes to Northeast
Princeton supplies 650000.00 units of Wipes to Southeast
LosAngeles supplies 675000.00 units of Wipes to Northwest
LosAngeles supplies 904500.00 u